In [237]:
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import swifter
from fastai.text import *
path = Path('/home/jupyter/tutorials/Thathasthu')
import gensim
from spellchecker import SpellChecker
spell = SpellChecker()

In [112]:
!pip install boto

In [205]:

train = pd.read_csv('./tathasthu/ttrain.csv')
manufacturers = pd.read_csv("./tathasthu/manufacturer_list.csv")


In [206]:
def clean_column(column, delimitter_list):
    column = column.str.lower()
    for delimitter in delimitter_list: 
        column = column.str.replace(delimitter,"")
    column = column.str.replace("  "," ")
    column = column.str.replace("  "," ")
    column = column.str.replace("  "," ")
    column = column.str.replace("  "," ")
    column = column.str.replace("  "," ")
    column = column.str.replace("  "," ")
    return column

def get_brand_name(article_name, choices):
    maxScore = 0
    maxChoice = ""
    for choice in choices:
        score = fuzz.token_set_ratio(" " + article_name + " ", " " + choice + " ")
        if score > maxScore or (score == maxScore and len(choice) > len(maxChoice)):
            maxScore = score
            maxChoice = choice
        if len(choice) > 5:
            concat_article_name = article_name.replace(" ", "")
            concat_choice = choice.replace(" ", "")
            score = 100 if concat_choice in concat_article_name else 0
            if score > maxScore or (score == maxScore and len(choice) > len(maxChoice)):
                maxScore = score
                maxChoice = choice
    return maxChoice, maxScore

def get_strict(article_name, choices):
    matches = []
    for choice in choices:
        choice1 = choice + " "
        choice2 = " " + choice + " "
        choice3 = " " + choice
        if article_name.startswith(choice1) or article_name.endswith(choice3) or choice2 in article_name:
            matches.append(choice)
    return matches

def remove_string(large, queries):
    for query in queries.split(" "):
        large = large.replace(query, "")
        large = large.replace(query+"s", "")
    large = large.replace("  ", " ")
    large = large.replace("  ", " ")
    large = large.replace("  ", " ")
    return large

def get_type(filtered_string):
    type_column = set()
    for word in filtered_string.split(' '):
        for seed in seed_choices:
            try: 
                similarity_score = model.similarity(seed, word)
            except KeyError:
                similarity_score = 0.0
            if similarity_score > 0.35:
                type_column.add(word)
    #         print(seed + " " + word + " " + str(similarity_score))
    s = list(type_column)
    string_array = ' '.join(str(e) for e in s)
    return string_array

def get_words_from_sentence(sentence):
    words = sentence.split(" ")
    returns = []
    misspelled = spell.unknown(words)
    for word in words:
        if len(word) > 2 and word not in misspelled and word not in stopwords:
            returns.append(word)
    return returns

seed_choices = ['oranges', 'apples', 'avocado', 'onion', 'tomato', 'pants', 'shirt', 'pink', 'blue']
seed_colors = ['red', 'blue', 'green']
seed_flavors = ['onion', 'tomato', 'apples', 'avocado']
seed_type = ['shirt', 'pant', 'soap', 'cosmetics']

def get_type_specific(filtered_string):
    type_column = set()
    for word in filtered_string.split(' '):
        for seed in seed_colors:
            try:
                similarity_score = model.similarity(seed, word)
            except KeyError:
                similarity_score = 0.0
            if similarity_score > 0.35:
                 type_column.add('color:' + word)
       
        for seed in seed_flavors:
            try:
                similarity_score = model.similarity(seed, word)
            except KeyError:
                similarity_score = 0.0
            if similarity_score > 0.35:
                type_column.add('flavor:' + word)

        for seed in seed_type:
            try:
                similarity_score = model.similarity(seed, word)
            except KeyError:
                similarity_score = 0.0
            if similarity_score > 0.35:
                type_column.add('type:' + word)
    
    s = list(type_column)
    string_array = ' '.join(str(e) for e in s)
    return string_array

In [207]:
train['manufacturer_description'] = clean_column(train['manufacturer_description'], [])

In [208]:
train["company_name"] = train.manufacturer_description.str.extract(r"company name - ([a-zA-Z-09\. \&]+)")

In [209]:
train["article_name"] = clean_column(train['article_name'], ["'", "(", ")", "/", "-"])

In [210]:
train['quantity'] = train['article_name'].str.extract(r'(\d.{0,4}(packs|m|ml|g|kg|l|s|p))')[0]

In [211]:
manufacturers["brand_name"] = clean_column(manufacturers['brand_name'], ["'", "(", ")", "/", "-"])

In [212]:
train["company_name"] = train["company_name"].fillna("")

In [213]:
train["article_name"] = train["article_name"].fillna("")

In [214]:
train['strict_transformed'] = train['article_name'].apply(lambda x: get_strict(x, manufacturers['brand_name']))

In [215]:
train['strict_company'] = train['company_name'].apply(lambda x: get_strict(x, manufacturers['brand_name']))

In [216]:
train = train[train["article_name"] != ""]

In [217]:
train2 = train

In [218]:
train['fuzzy_transformed'] = train['article_name'].swifter.apply(lambda x: get_brand_name(x, manufacturers['brand_name']))




Pandas Apply:   0%|          | 0/8621 [00:00<?, ?it/s]


Pandas Apply:   0%|          | 7/8621 [00:00<02:05, 68.54it/s]


Pandas Apply:   0%|          | 13/8621 [00:00<02:14, 64.09it/s]


Pandas Apply:   0%|          | 19/8621 [00:00<02:20, 61.38it/s]


Pandas Apply:   0%|          | 25/8621 [00:00<02:22, 60.16it/s]


Pandas Apply:   0%|          | 32/8621 [00:00<02:21, 60.67it/s]


Pandas Apply:   0%|          | 39/8621 [00:00<02:17, 62.37it/s]


Pandas Apply:   1%|          | 46/8621 [00:00<02:16, 62.91it/s]


Pandas Apply:   1%|          | 53/8621 [00:00<02:15, 63.02it/s]


Pandas Apply:   1%|          | 60/8621 [00:00<02:17, 62.39it/s]


Pandas Apply:   1%|          | 67/8621 [00:01<02:16, 62.64it/s]


Pandas Apply:   1%|          | 74/8621 [00:01<02:15, 63.05it/s]


Pandas Apply:   1%|          | 81/8621 [00:01<02:15, 62.94it/s]


Pandas Apply:   1%|          | 88/8621 [00:01<02:14, 63.21it/s]


Pandas Apply:   1%|          | 95/8621 [00:01<02:14, 63.21it/s]


Pandas Apply:   1

Pandas Apply:   9%|▊         | 754/8621 [00:11<02:03, 63.69it/s]


Pandas Apply:   9%|▉         | 761/8621 [00:11<02:03, 63.72it/s]


Pandas Apply:   9%|▉         | 768/8621 [00:11<02:02, 63.86it/s]


Pandas Apply:   9%|▉         | 775/8621 [00:12<02:03, 63.77it/s]


Pandas Apply:   9%|▉         | 782/8621 [00:12<02:04, 63.00it/s]


Pandas Apply:   9%|▉         | 789/8621 [00:12<02:05, 62.19it/s]


Pandas Apply:   9%|▉         | 796/8621 [00:12<02:05, 62.35it/s]


Pandas Apply:   9%|▉         | 803/8621 [00:12<02:04, 62.82it/s]


Pandas Apply:   9%|▉         | 810/8621 [00:12<02:03, 63.00it/s]


Pandas Apply:   9%|▉         | 817/8621 [00:12<02:02, 63.64it/s]


Pandas Apply:  10%|▉         | 825/8621 [00:12<01:59, 65.36it/s]


Pandas Apply:  10%|▉         | 833/8621 [00:12<01:56, 67.13it/s]


Pandas Apply:  10%|▉         | 840/8621 [00:13<01:56, 66.52it/s]


Pandas Apply:  10%|▉         | 847/8621 [00:13<01:57, 66.29it/s]


Pandas Apply:  10%|▉         | 854/8621 [00:13<01:58, 65.35it/

Pandas Apply:  17%|█▋        | 1504/8621 [00:23<01:49, 65.21it/s]


Pandas Apply:  18%|█▊        | 1511/8621 [00:23<01:47, 66.17it/s]


Pandas Apply:  18%|█▊        | 1518/8621 [00:23<01:46, 66.47it/s]


Pandas Apply:  18%|█▊        | 1525/8621 [00:23<01:46, 66.70it/s]


Pandas Apply:  18%|█▊        | 1533/8621 [00:23<01:45, 67.28it/s]


Pandas Apply:  18%|█▊        | 1540/8621 [00:23<01:49, 64.48it/s]


Pandas Apply:  18%|█▊        | 1547/8621 [00:23<01:49, 64.72it/s]


Pandas Apply:  18%|█▊        | 1554/8621 [00:23<01:49, 64.48it/s]


Pandas Apply:  18%|█▊        | 1561/8621 [00:24<01:51, 63.13it/s]


Pandas Apply:  18%|█▊        | 1568/8621 [00:24<01:50, 63.89it/s]


Pandas Apply:  18%|█▊        | 1575/8621 [00:24<01:49, 64.12it/s]


Pandas Apply:  18%|█▊        | 1582/8621 [00:24<01:48, 64.65it/s]


Pandas Apply:  18%|█▊        | 1589/8621 [00:24<01:47, 65.18it/s]


Pandas Apply:  19%|█▊        | 1596/8621 [00:24<01:48, 64.87it/s]


Pandas Apply:  19%|█▊        | 1603/8621 [00:24<

Pandas Apply:  26%|██▌       | 2257/8621 [00:34<01:35, 66.86it/s]


Pandas Apply:  26%|██▋       | 2264/8621 [00:34<01:34, 67.52it/s]


Pandas Apply:  26%|██▋       | 2271/8621 [00:34<01:35, 66.64it/s]


Pandas Apply:  26%|██▋       | 2278/8621 [00:34<01:36, 65.58it/s]


Pandas Apply:  27%|██▋       | 2285/8621 [00:35<01:37, 64.95it/s]


Pandas Apply:  27%|██▋       | 2292/8621 [00:35<01:38, 64.17it/s]


Pandas Apply:  27%|██▋       | 2299/8621 [00:35<01:37, 64.98it/s]


Pandas Apply:  27%|██▋       | 2306/8621 [00:35<01:35, 66.18it/s]


Pandas Apply:  27%|██▋       | 2313/8621 [00:35<01:35, 65.76it/s]


Pandas Apply:  27%|██▋       | 2320/8621 [00:35<01:36, 65.42it/s]


Pandas Apply:  27%|██▋       | 2327/8621 [00:35<01:37, 64.58it/s]


Pandas Apply:  27%|██▋       | 2334/8621 [00:35<01:37, 64.51it/s]


Pandas Apply:  27%|██▋       | 2341/8621 [00:35<01:37, 64.15it/s]


Pandas Apply:  27%|██▋       | 2348/8621 [00:35<01:37, 64.23it/s]


Pandas Apply:  27%|██▋       | 2355/8621 [00:36<

Pandas Apply:  35%|███▍      | 3003/8621 [00:45<01:24, 66.53it/s]


Pandas Apply:  35%|███▍      | 3010/8621 [00:46<01:23, 67.24it/s]


Pandas Apply:  35%|███▍      | 3017/8621 [00:46<01:24, 66.09it/s]


Pandas Apply:  35%|███▌      | 3024/8621 [00:46<01:24, 66.47it/s]


Pandas Apply:  35%|███▌      | 3031/8621 [00:46<01:23, 66.56it/s]


Pandas Apply:  35%|███▌      | 3038/8621 [00:46<01:24, 65.91it/s]


Pandas Apply:  35%|███▌      | 3045/8621 [00:46<01:25, 65.28it/s]


Pandas Apply:  35%|███▌      | 3052/8621 [00:46<01:25, 64.99it/s]


Pandas Apply:  35%|███▌      | 3059/8621 [00:46<01:25, 64.73it/s]


Pandas Apply:  36%|███▌      | 3066/8621 [00:46<01:24, 65.85it/s]


Pandas Apply:  36%|███▌      | 3073/8621 [00:47<01:23, 66.05it/s]


Pandas Apply:  36%|███▌      | 3080/8621 [00:47<01:24, 65.47it/s]


Pandas Apply:  36%|███▌      | 3087/8621 [00:47<01:24, 65.38it/s]


Pandas Apply:  36%|███▌      | 3094/8621 [00:47<01:23, 66.21it/s]


Pandas Apply:  36%|███▌      | 3101/8621 [00:47<

...

Pandas Apply:  60%|█████▉    | 5154/8621 [01:20<00:52, 66.60it/s]


Pandas Apply:  60%|█████▉    | 5161/8621 [01:20<00:51, 66.61it/s]


Pandas Apply:  60%|█████▉    | 5168/8621 [01:20<00:52, 65.56it/s]


Pandas Apply:  60%|██████    | 5175/8621 [01:20<00:53, 64.59it/s]


Pandas Apply:  60%|██████    | 5182/8621 [01:20<00:52, 65.03it/s]


Pandas Apply:  60%|██████    | 5189/8621 [01:20<00:52, 65.08it/s]


Pandas Apply:  60%|██████    | 5196/8621 [01:20<00:52, 64.98it/s]


Pandas Apply:  60%|██████    | 5203/8621 [01:20<00:52, 64.60it/s]


Pandas Apply:  60%|██████    | 5211/8621 [01:20<00:51, 66.02it/s]


Pandas Apply:  61%|██████    | 5218/8621 [01:21<00:51, 66.45it/s]


Pandas Apply:  61%|██████    | 5225/8621 [01:21<00:52, 65.28it/s]


Pandas Apply:  61%|██████    | 5232/8621 [01:21<00:51, 65.98it/s]


Pandas Apply:  61%|██████    | 5240/8621 [01:21<00:50, 67.43it/s]


Pandas Apply:  61%|██████    | 5247/8621 [01:21<00:49, 67.56it/s]


Pandas Apply:  61%|██████    | 5254/8621 [01:21<

Pandas Apply:  68%|██████▊   | 5903/8621 [01:31<00:41, 65.23it/s]


Pandas Apply:  69%|██████▊   | 5910/8621 [01:31<00:41, 64.92it/s]


Pandas Apply:  69%|██████▊   | 5917/8621 [01:31<00:41, 64.82it/s]


Pandas Apply:  69%|██████▊   | 5925/8621 [01:31<00:39, 68.41it/s]


Pandas Apply:  69%|██████▉   | 5934/8621 [01:31<00:37, 71.44it/s]


Pandas Apply:  69%|██████▉   | 5942/8621 [01:31<00:38, 70.14it/s]


Pandas Apply:  69%|██████▉   | 5950/8621 [01:32<00:39, 68.39it/s]


Pandas Apply:  69%|██████▉   | 5957/8621 [01:32<00:39, 67.60it/s]


Pandas Apply:  69%|██████▉   | 5964/8621 [01:32<00:40, 65.85it/s]


Pandas Apply:  69%|██████▉   | 5971/8621 [01:32<00:40, 65.13it/s]


Pandas Apply:  69%|██████▉   | 5978/8621 [01:32<00:40, 65.55it/s]


Pandas Apply:  69%|██████▉   | 5985/8621 [01:32<00:40, 65.52it/s]


Pandas Apply:  70%|██████▉   | 5992/8621 [01:32<00:40, 64.99it/s]


Pandas Apply:  70%|██████▉   | 5999/8621 [01:32<00:40, 64.95it/s]


Pandas Apply:  70%|██████▉   | 6006/8621 [01:32<

Pandas Apply:  77%|███████▋  | 6654/8621 [01:42<00:29, 67.63it/s]


Pandas Apply:  77%|███████▋  | 6661/8621 [01:42<00:29, 66.47it/s]


Pandas Apply:  77%|███████▋  | 6668/8621 [01:43<00:29, 66.11it/s]


Pandas Apply:  77%|███████▋  | 6675/8621 [01:43<00:29, 66.08it/s]


Pandas Apply:  78%|███████▊  | 6682/8621 [01:43<00:29, 66.38it/s]


Pandas Apply:  78%|███████▊  | 6689/8621 [01:43<00:29, 66.20it/s]


Pandas Apply:  78%|███████▊  | 6696/8621 [01:43<00:29, 65.86it/s]


Pandas Apply:  78%|███████▊  | 6703/8621 [01:43<00:28, 66.54it/s]


Pandas Apply:  78%|███████▊  | 6710/8621 [01:43<00:28, 66.08it/s]


Pandas Apply:  78%|███████▊  | 6717/8621 [01:43<00:28, 65.76it/s]


Pandas Apply:  78%|███████▊  | 6724/8621 [01:43<00:28, 66.15it/s]


Pandas Apply:  78%|███████▊  | 6731/8621 [01:43<00:28, 65.69it/s]


Pandas Apply:  78%|███████▊  | 6738/8621 [01:44<00:28, 65.78it/s]


Pandas Apply:  78%|███████▊  | 6745/8621 [01:44<00:28, 66.28it/s]


Pandas Apply:  78%|███████▊  | 6752/8621 [01:44<

Pandas Apply:  86%|████████▌ | 7398/8621 [01:54<00:19, 62.06it/s]


Pandas Apply:  86%|████████▌ | 7406/8621 [01:54<00:18, 65.73it/s]


Pandas Apply:  86%|████████▌ | 7414/8621 [01:54<00:17, 67.63it/s]


Pandas Apply:  86%|████████▌ | 7421/8621 [01:54<00:17, 67.29it/s]


Pandas Apply:  86%|████████▌ | 7428/8621 [01:54<00:17, 67.03it/s]


Pandas Apply:  86%|████████▌ | 7435/8621 [01:54<00:18, 65.42it/s]


Pandas Apply:  86%|████████▋ | 7442/8621 [01:54<00:18, 64.25it/s]


Pandas Apply:  86%|████████▋ | 7449/8621 [01:55<00:18, 64.02it/s]


Pandas Apply:  86%|████████▋ | 7456/8621 [01:55<00:18, 64.68it/s]


Pandas Apply:  87%|████████▋ | 7463/8621 [01:55<00:17, 65.36it/s]


Pandas Apply:  87%|████████▋ | 7470/8621 [01:55<00:17, 65.35it/s]


Pandas Apply:  87%|████████▋ | 7477/8621 [01:55<00:17, 64.86it/s]


Pandas Apply:  87%|████████▋ | 7484/8621 [01:55<00:17, 64.40it/s]


Pandas Apply:  87%|████████▋ | 7491/8621 [01:55<00:17, 65.61it/s]


Pandas Apply:  87%|████████▋ | 7498/8621 [01:55<

Pandas Apply:  94%|█████████▍| 8146/8621 [02:05<00:07, 66.66it/s]


Pandas Apply:  95%|█████████▍| 8153/8621 [02:05<00:06, 67.19it/s]


Pandas Apply:  95%|█████████▍| 8160/8621 [02:05<00:06, 67.96it/s]


Pandas Apply:  95%|█████████▍| 8167/8621 [02:05<00:06, 66.34it/s]


Pandas Apply:  95%|█████████▍| 8174/8621 [02:06<00:06, 65.96it/s]


Pandas Apply:  95%|█████████▍| 8181/8621 [02:06<00:06, 64.65it/s]


Pandas Apply:  95%|█████████▍| 8188/8621 [02:06<00:06, 65.08it/s]


Pandas Apply:  95%|█████████▌| 8195/8621 [02:06<00:06, 64.75it/s]


Pandas Apply:  95%|█████████▌| 8202/8621 [02:06<00:06, 64.63it/s]


Pandas Apply:  95%|█████████▌| 8209/8621 [02:06<00:06, 65.10it/s]


Pandas Apply:  95%|█████████▌| 8216/8621 [02:06<00:06, 63.30it/s]


Pandas Apply:  95%|█████████▌| 8223/8621 [02:06<00:06, 63.78it/s]


Pandas Apply:  95%|█████████▌| 8230/8621 [02:06<00:06, 64.14it/s]


Pandas Apply:  96%|█████████▌| 8237/8621 [02:07<00:05, 64.46it/s]


Pandas Apply:  96%|█████████▌| 8244/8621 [02:07<

In [219]:
train['fuzzy_company'] = train['company_name'].swifter.apply(lambda x: get_brand_name(x, manufacturers['brand_name']))




Pandas Apply:   0%|          | 0/8621 [00:00<?, ?it/s]


Pandas Apply:   0%|          | 10/8621 [00:00<01:31, 93.81it/s]


Pandas Apply:   0%|          | 19/8621 [00:00<01:36, 89.41it/s]


Pandas Apply:   0%|          | 28/8621 [00:00<01:39, 86.30it/s]


Pandas Apply:   0%|          | 35/8621 [00:00<01:48, 79.21it/s]


Pandas Apply:   1%|          | 44/8621 [00:00<01:48, 79.21it/s]


Pandas Apply:   1%|          | 52/8621 [00:00<01:51, 77.17it/s]


Pandas Apply:   1%|          | 60/8621 [00:00<01:53, 75.30it/s]


Pandas Apply:   1%|          | 69/8621 [00:00<01:48, 78.53it/s]


Pandas Apply:   1%|          | 77/8621 [00:00<01:51, 76.71it/s]


Pandas Apply:   1%|          | 85/8621 [00:01<01:51, 76.68it/s]


Pandas Apply:   1%|          | 93/8621 [00:01<01:51, 76.72it/s]


Pandas Apply:   1%|          | 101/8621 [00:01<01:54, 74.35it/s]


Pandas Apply:   1%|▏         | 110/8621 [00:01<01:49, 77.57it/s]


Pandas Apply:   1%|▏         | 118/8621 [00:01<01:51, 76.47it/s]


Pandas Apply:

Pandas Apply:  10%|█         | 896/8621 [00:11<01:40, 77.18it/s]


Pandas Apply:  10%|█         | 904/8621 [00:11<01:39, 77.29it/s]


Pandas Apply:  11%|█         | 912/8621 [00:12<01:41, 75.69it/s]


Pandas Apply:  11%|█         | 920/8621 [00:12<01:42, 75.27it/s]


Pandas Apply:  11%|█         | 928/8621 [00:12<01:42, 74.73it/s]


Pandas Apply:  11%|█         | 936/8621 [00:12<01:44, 73.36it/s]


Pandas Apply:  11%|█         | 944/8621 [00:12<01:45, 72.63it/s]


Pandas Apply:  11%|█         | 952/8621 [00:12<01:44, 73.16it/s]


Pandas Apply:  11%|█         | 960/8621 [00:12<01:46, 72.18it/s]


Pandas Apply:  11%|█         | 968/8621 [00:12<01:46, 71.89it/s]


Pandas Apply:  11%|█▏        | 976/8621 [00:12<01:48, 70.20it/s]


Pandas Apply:  11%|█▏        | 984/8621 [00:13<01:48, 70.22it/s]


Pandas Apply:  12%|█▏        | 993/8621 [00:13<01:43, 73.58it/s]


Pandas Apply:  12%|█▏        | 1001/8621 [00:13<01:43, 73.70it/s]


Pandas Apply:  12%|█▏        | 1009/8621 [00:13<01:42, 74.24i

Pandas Apply:  21%|██        | 1769/8621 [00:23<01:27, 77.89it/s]


Pandas Apply:  21%|██        | 1777/8621 [00:23<01:30, 75.95it/s]


Pandas Apply:  21%|██        | 1785/8621 [00:23<01:30, 75.79it/s]


Pandas Apply:  21%|██        | 1793/8621 [00:23<01:31, 74.53it/s]


Pandas Apply:  21%|██        | 1801/8621 [00:23<01:30, 75.39it/s]


Pandas Apply:  21%|██        | 1810/8621 [00:23<01:26, 78.45it/s]


Pandas Apply:  21%|██        | 1819/8621 [00:24<01:26, 78.91it/s]


Pandas Apply:  21%|██        | 1827/8621 [00:24<01:27, 77.27it/s]


Pandas Apply:  21%|██▏       | 1835/8621 [00:24<01:29, 76.01it/s]


Pandas Apply:  21%|██▏       | 1843/8621 [00:24<01:31, 73.85it/s]


Pandas Apply:  21%|██▏       | 1851/8621 [00:24<01:31, 74.36it/s]


Pandas Apply:  22%|██▏       | 1859/8621 [00:24<01:33, 72.52it/s]


Pandas Apply:  22%|██▏       | 1867/8621 [00:24<01:33, 71.92it/s]


Pandas Apply:  22%|██▏       | 1876/8621 [00:24<01:28, 75.87it/s]


Pandas Apply:  22%|██▏       | 1885/8621 [00:24<

Pandas Apply:  30%|██▉       | 2582/8621 [00:34<01:19, 75.83it/s]


Pandas Apply:  30%|███       | 2590/8621 [00:35<01:20, 74.85it/s]


Pandas Apply:  30%|███       | 2598/8621 [00:35<01:21, 74.25it/s]


Pandas Apply:  30%|███       | 2606/8621 [00:35<01:21, 73.88it/s]


Pandas Apply:  30%|███       | 2614/8621 [00:35<01:21, 73.63it/s]


Pandas Apply:  30%|███       | 2624/8621 [00:35<01:17, 77.74it/s]


Pandas Apply:  31%|███       | 2633/8621 [00:35<01:14, 80.56it/s]


Pandas Apply:  31%|███       | 2642/8621 [00:35<01:14, 80.49it/s]


Pandas Apply:  31%|███       | 2651/8621 [00:35<01:16, 78.40it/s]


Pandas Apply:  31%|███       | 2659/8621 [00:35<01:19, 75.21it/s]


Pandas Apply:  31%|███       | 2668/8621 [00:36<01:16, 78.25it/s]


Pandas Apply:  31%|███       | 2676/8621 [00:36<01:18, 75.84it/s]


Pandas Apply:  31%|███       | 2684/8621 [00:36<01:20, 73.99it/s]


Pandas Apply:  31%|███       | 2692/8621 [00:36<01:22, 72.12it/s]


Pandas Apply:  31%|███▏      | 2700/8621 [00:36<

Pandas Apply:  40%|████      | 3451/8621 [00:46<01:16, 67.96it/s]


Pandas Apply:  40%|████      | 3458/8621 [00:46<01:17, 66.65it/s]


Pandas Apply:  40%|████      | 3465/8621 [00:46<01:16, 67.21it/s]


Pandas Apply:  40%|████      | 3472/8621 [00:46<01:18, 65.97it/s]


Pandas Apply:  40%|████      | 3480/8621 [00:47<01:14, 68.57it/s]


Pandas Apply:  40%|████      | 3487/8621 [00:47<01:16, 67.53it/s]


Pandas Apply:  41%|████      | 3494/8621 [00:47<01:18, 65.36it/s]


Pandas Apply:  41%|████      | 3501/8621 [00:47<01:18, 65.13it/s]


Pandas Apply:  41%|████      | 3508/8621 [00:47<01:18, 65.12it/s]


Pandas Apply:  41%|████      | 3515/8621 [00:47<01:18, 65.13it/s]


Pandas Apply:  41%|████      | 3522/8621 [00:47<01:16, 66.36it/s]


Pandas Apply:  41%|████      | 3530/8621 [00:47<01:13, 68.82it/s]


Pandas Apply:  41%|████      | 3538/8621 [00:47<01:12, 70.45it/s]


Pandas Apply:  41%|████      | 3546/8621 [00:48<01:14, 68.56it/s]


Pandas Apply:  41%|████      | 3553/8621 [00:48<

Pandas Apply:  49%|████▉     | 4252/8621 [00:58<00:57, 75.41it/s]


Pandas Apply:  49%|████▉     | 4260/8621 [00:58<00:58, 74.69it/s]


Pandas Apply:  50%|████▉     | 4270/8621 [00:58<00:55, 78.70it/s]


Pandas Apply:  50%|████▉     | 4278/8621 [00:58<00:55, 78.65it/s]


Pandas Apply:  50%|████▉     | 4286/8621 [00:58<00:56, 77.38it/s]


Pandas Apply:  50%|████▉     | 4294/8621 [00:58<00:56, 76.03it/s]


Pandas Apply:  50%|████▉     | 4302/8621 [00:58<00:58, 74.33it/s]


Pandas Apply:  50%|████▉     | 4310/8621 [00:59<00:57, 75.11it/s]


Pandas Apply:  50%|█████     | 4318/8621 [00:59<00:58, 73.93it/s]


Pandas Apply:  50%|█████     | 4326/8621 [00:59<00:57, 74.26it/s]


Pandas Apply:  50%|█████     | 4336/8621 [00:59<00:54, 79.29it/s]


Pandas Apply:  50%|█████     | 4345/8621 [00:59<00:53, 79.54it/s]


Pandas Apply:  51%|█████     | 4354/8621 [00:59<00:52, 80.78it/s]


Pandas Apply:  51%|█████     | 4363/8621 [00:59<00:52, 81.80it/s]


Pandas Apply:  51%|█████     | 4372/8621 [00:59<

Pandas Apply:  60%|█████▉    | 5152/8621 [01:09<00:45, 75.81it/s]


Pandas Apply:  60%|█████▉    | 5160/8621 [01:10<00:44, 77.00it/s]


Pandas Apply:  60%|█████▉    | 5168/8621 [01:10<00:44, 77.31it/s]


Pandas Apply:  60%|██████    | 5176/8621 [01:10<00:44, 76.83it/s]


Pandas Apply:  60%|██████    | 5184/8621 [01:10<00:44, 77.75it/s]


Pandas Apply:  60%|██████    | 5192/8621 [01:10<00:46, 74.42it/s]


Pandas Apply:  60%|██████    | 5200/8621 [01:10<00:46, 74.33it/s]


Pandas Apply:  60%|██████    | 5208/8621 [01:10<00:45, 74.98it/s]


Pandas Apply:  61%|██████    | 5216/8621 [01:10<00:46, 73.45it/s]


Pandas Apply:  61%|██████    | 5225/8621 [01:10<00:44, 75.75it/s]


Pandas Apply:  61%|██████    | 5233/8621 [01:10<00:44, 75.72it/s]


Pandas Apply:  61%|██████    | 5244/8621 [01:11<00:41, 81.28it/s]


Pandas Apply:  61%|██████    | 5253/8621 [01:11<00:41, 81.07it/s]


Pandas Apply:  61%|██████    | 5262/8621 [01:11<00:42, 79.96it/s]


Pandas Apply:  61%|██████    | 5271/8621 [01:11<

Pandas Apply:  70%|███████   | 6039/8621 [01:21<00:34, 75.90it/s]


Pandas Apply:  70%|███████   | 6047/8621 [01:21<00:34, 74.82it/s]


Pandas Apply:  70%|███████   | 6055/8621 [01:21<00:34, 74.72it/s]


Pandas Apply:  70%|███████   | 6063/8621 [01:21<00:34, 74.49it/s]


Pandas Apply:  70%|███████   | 6071/8621 [01:21<00:34, 73.85it/s]


Pandas Apply:  71%|███████   | 6079/8621 [01:22<00:34, 73.23it/s]


Pandas Apply:  71%|███████   | 6088/8621 [01:22<00:33, 75.06it/s]


Pandas Apply:  71%|███████   | 6096/8621 [01:22<00:33, 75.80it/s]


Pandas Apply:  71%|███████   | 6104/8621 [01:22<00:33, 75.27it/s]


Pandas Apply:  71%|███████   | 6112/8621 [01:22<00:33, 73.84it/s]


Pandas Apply:  71%|███████   | 6120/8621 [01:22<00:33, 74.75it/s]


Pandas Apply:  71%|███████   | 6128/8621 [01:22<00:33, 74.73it/s]


Pandas Apply:  71%|███████   | 6136/8621 [01:22<00:33, 73.19it/s]


Pandas Apply:  71%|███████▏  | 6144/8621 [01:22<00:33, 73.14it/s]


Pandas Apply:  71%|███████▏  | 6152/8621 [01:23<

Pandas Apply:  80%|████████  | 6908/8621 [01:33<00:23, 73.05it/s]


Pandas Apply:  80%|████████  | 6916/8621 [01:33<00:22, 74.55it/s]


Pandas Apply:  80%|████████  | 6925/8621 [01:33<00:21, 78.23it/s]


Pandas Apply:  80%|████████  | 6934/8621 [01:33<00:21, 78.72it/s]


Pandas Apply:  81%|████████  | 6942/8621 [01:33<00:21, 77.00it/s]


Pandas Apply:  81%|████████  | 6950/8621 [01:33<00:22, 75.69it/s]


Pandas Apply:  81%|████████  | 6958/8621 [01:33<00:22, 75.03it/s]


Pandas Apply:  81%|████████  | 6966/8621 [01:33<00:21, 76.13it/s]


Pandas Apply:  81%|████████  | 6974/8621 [01:33<00:21, 76.24it/s]


Pandas Apply:  81%|████████  | 6982/8621 [01:33<00:21, 74.67it/s]


Pandas Apply:  81%|████████  | 6990/8621 [01:34<00:22, 73.75it/s]


Pandas Apply:  81%|████████  | 6998/8621 [01:34<00:21, 75.48it/s]


Pandas Apply:  81%|████████▏ | 7006/8621 [01:34<00:21, 74.97it/s]


Pandas Apply:  81%|████████▏ | 7014/8621 [01:34<00:21, 76.08it/s]


Pandas Apply:  81%|████████▏ | 7022/8621 [01:34<

Pandas Apply:  90%|█████████ | 7776/8621 [01:44<00:11, 75.55it/s]


Pandas Apply:  90%|█████████ | 7784/8621 [01:44<00:11, 74.45it/s]


Pandas Apply:  90%|█████████ | 7792/8621 [01:44<00:11, 74.09it/s]


Pandas Apply:  90%|█████████ | 7800/8621 [01:44<00:11, 72.62it/s]


Pandas Apply:  91%|█████████ | 7808/8621 [01:44<00:11, 72.80it/s]


Pandas Apply:  91%|█████████ | 7816/8621 [01:45<00:10, 74.27it/s]


Pandas Apply:  91%|█████████ | 7824/8621 [01:45<00:10, 73.83it/s]


Pandas Apply:  91%|█████████ | 7832/8621 [01:45<00:10, 73.06it/s]


Pandas Apply:  91%|█████████ | 7840/8621 [01:45<00:10, 72.97it/s]


Pandas Apply:  91%|█████████ | 7848/8621 [01:45<00:10, 74.47it/s]


Pandas Apply:  91%|█████████ | 7856/8621 [01:45<00:10, 74.37it/s]


Pandas Apply:  91%|█████████ | 7865/8621 [01:45<00:09, 75.99it/s]


Pandas Apply:  91%|█████████▏| 7873/8621 [01:45<00:10, 74.79it/s]


Pandas Apply:  91%|█████████▏| 7882/8621 [01:45<00:09, 76.53it/s]


Pandas Apply:  92%|█████████▏| 7890/8621 [01:46<

In [220]:
train = train.filter(['article_name','company_name', 'quantity', 'strict_transformed', 'strict_company', 'fuzzy_transformed', 'fuzzy_company']).join(train.fuzzy_transformed.apply(lambda loc: pd.Series(loc, index=['fuzzy_brand', 'brand_score']))).join(train.fuzzy_company.apply(lambda loc: pd.Series(loc, index=['f_company', 'company_score'])))



In [221]:
del train['fuzzy_transformed']
del train['fuzzy_company']

In [222]:
def get_brand_from_map(article_name):
    brand_map = {
        "ph": "premium harvest",
        "ttrt": "tasty treat",
        "gh": "golden harvest",
        "on1y": "only"
    }
    for key in brand_map:
        if key in article_name:
            return brand_map[key]
    return None

In [223]:
train3 = train

In [224]:
train

,article_name,company_name,quantity,strict_transformed,strict_company,fuzzy_brand,brand_score,f_company,company_score
0,chicken burger patty 250g,venkys india ltd,250g,[],"[venkys, venkys]",easy choice,44,venkys,100
1,britannia daily family long bread,britannia industries ltd,NaN,[britannia],[britannia],britannia,100,britannia,100
2,pooof potato kettle chips zesty masala pp 30g,future consumer enterprise limited,30g,[],[],pooof!,100,center fruit,48
3,fevibond tube 20 ml,,20 ml,[fevibond],[],fevibond,100,emami fair and handsome,0
4,kitkat dessert delight truffle 50g,nestle india ltd,50g,[],[nestle],tastytreat disney,43,nestle,100
5,blue bird spaghetti pasta 200g,blue bird foods india pvt ltd,200g,[blue bird],[blue bird],blue bird,100,blue bird,100
6,nirapara avalose podi 500gm,k k r food products,500gm,[nirapara],[],nirapara,100,hr foods,56
7,nivea roll on fresh natural 50ml,,50ml,[nivea],[],nivea,100,emami fair and handsome,0
8,fiama dw soap exotic dream bx 115g,itc india ltd,115g,[],[],fiama di wills,53,kitchens of india,67
9,sangis ktcn peri peri mayonnaise bt 200g,future consumer ltd,200g,[],[],sangis kitchen,60,center fruit,52


In [225]:
train["final_brand"] = train.swifter.apply(lambda x: x.fuzzy_brand if x.brand_score > 50 else x.strict_company[0] if len(x.strict_company) > 0 else get_brand_from_map(x.article_name), axis=1)





Pandas Apply:   0%|          | 0/8621 [00:00<?, ?it/s]


Pandas Apply:  48%|████▊     | 4120/8621 [00:00<00:00, 41199.84it/s]


Pandas Apply:  93%|█████████▎| 8026/8621 [00:00<00:00, 40531.00it/s]


Pandas Apply: 100%|██████████| 8621/8621 [00:00<00:00, 39071.57it/s]

In [226]:
train = train.filter(["article_name", "quantity", "final_brand"])

In [227]:
train["filtered"] = train.apply(lambda x: x.article_name if x.final_brand is None else remove_string(x.article_name, x.final_brand), axis=1)


In [228]:
train["filtered"] = train.apply(lambda x: remove_string(x.filtered, str(x.quantity)), axis=1)

In [229]:
train.to_csv("train_quantity.csv", index=False)

In [233]:
train = pd.read_csv("train_quantity.csv")

In [234]:
learn = load_learner(path)

In [235]:
train["final_brand"] = train.swifter.apply(lambda x: "" if x.final_brand == None else x.final_brand, axis=1)





Pandas Apply:   0%|          | 0/8621 [00:00<?, ?it/s]



Pandas Apply:  52%|█████▏    | 4501/8621 [00:00<00:00, 45000.17it/s]



Pandas Apply: 100%|██████████| 8621/8621 [00:00<00:00, 44927.34it/s]

In [239]:
train['final_brand'] = train['final_brand'].fillna('')

In [240]:
train['filtered'] = train['filtered'].fillna('')

In [241]:
train['function_dl'] = train.swifter.apply(lambda x: str(learn.predict(x['final_brand']+' '+x['filtered'])[0]), axis=1)





Pandas Apply:   0%|          | 0/8621 [00:00<?, ?it/s]



Pandas Apply:   0%|          | 3/8621 [00:00<04:50, 29.64it/s]



Pandas Apply:   0%|          | 6/8621 [00:00<05:25, 26.50it/s]



Pandas Apply:   0%|          | 9/8621 [00:00<05:46, 24.89it/s]



Pandas Apply:   0%|          | 12/8621 [00:00<06:03, 23.66it/s]



Pandas Apply:   0%|          | 15/8621 [00:00<06:15, 22.91it/s]



Pandas Apply:   0%|          | 18/8621 [00:00<06:21, 22.56it/s]



Pandas Apply:   0%|          | 21/8621 [00:00<06:27, 22.18it/s]



Pandas Apply:   0%|          | 24/8621 [00:01<06:34, 21.79it/s]



Pandas Apply:   0%|          | 27/8621 [00:01<06:45, 21.20it/s]



Pandas Apply:   0%|          | 30/8621 [00:01<06:43, 21.29it/s]



Pandas Apply:   0%|          | 33/8621 [00:01<06:38, 21.53it/s]



Pandas Apply:   0%|          | 36/8621 [00:01<06:34, 21.74it/s]



Pandas Apply:   0%|          | 39/8621 [00:01<06:32, 21.87it/s]



Pandas Apply:   0%|          | 42/8621 [00:01<06:33, 21.80it/s]



Pan

Pandas Apply:   4%|▎         | 306/8621 [00:14<06:19, 21.88it/s]



Pandas Apply:   4%|▎         | 309/8621 [00:14<06:18, 21.95it/s]



Pandas Apply:   4%|▎         | 312/8621 [00:14<06:18, 21.95it/s]



Pandas Apply:   4%|▎         | 315/8621 [00:14<06:18, 21.96it/s]



Pandas Apply:   4%|▎         | 318/8621 [00:14<06:17, 22.00it/s]



Pandas Apply:   4%|▎         | 321/8621 [00:14<06:21, 21.74it/s]



Pandas Apply:   4%|▍         | 324/8621 [00:14<06:23, 21.63it/s]



Pandas Apply:   4%|▍         | 327/8621 [00:15<06:23, 21.61it/s]



Pandas Apply:   4%|▍         | 330/8621 [00:15<06:22, 21.68it/s]



Pandas Apply:   4%|▍         | 333/8621 [00:15<06:20, 21.78it/s]



Pandas Apply:   4%|▍         | 336/8621 [00:15<06:18, 21.90it/s]



Pandas Apply:   4%|▍         | 339/8621 [00:15<06:18, 21.88it/s]



Pandas Apply:   4%|▍         | 342/8621 [00:15<06:17, 21.93it/s]



Pandas Apply:   4%|▍         | 345/8621 [00:15<06:18, 21.89it/s]



Pandas Apply:   4%|▍         | 348/8621 [00:16<0

Pandas Apply:   7%|▋         | 599/8621 [00:28<06:21, 21.02it/s]



Pandas Apply:   7%|▋         | 602/8621 [00:28<06:28, 20.66it/s]



Pandas Apply:   7%|▋         | 605/8621 [00:28<06:28, 20.63it/s]



Pandas Apply:   7%|▋         | 608/8621 [00:28<06:28, 20.64it/s]



Pandas Apply:   7%|▋         | 611/8621 [00:28<06:33, 20.33it/s]



Pandas Apply:   7%|▋         | 614/8621 [00:29<06:35, 20.26it/s]



Pandas Apply:   7%|▋         | 617/8621 [00:29<06:34, 20.27it/s]



Pandas Apply:   7%|▋         | 620/8621 [00:29<06:36, 20.19it/s]



Pandas Apply:   7%|▋         | 623/8621 [00:29<06:32, 20.38it/s]



Pandas Apply:   7%|▋         | 626/8621 [00:29<06:31, 20.41it/s]



Pandas Apply:   7%|▋         | 629/8621 [00:29<06:32, 20.38it/s]



Pandas Apply:   7%|▋         | 632/8621 [00:29<06:33, 20.32it/s]



Pandas Apply:   7%|▋         | 635/8621 [00:30<06:30, 20.46it/s]



Pandas Apply:   7%|▋         | 638/8621 [00:30<06:31, 20.40it/s]



Pandas Apply:   7%|▋         | 641/8621 [00:30<0

Pandas Apply:  10%|█         | 895/8621 [00:42<06:20, 20.32it/s]



Pandas Apply:  10%|█         | 898/8621 [00:42<06:24, 20.08it/s]



Pandas Apply:  10%|█         | 901/8621 [00:42<06:26, 19.97it/s]



Pandas Apply:  10%|█         | 904/8621 [00:42<06:26, 19.95it/s]



Pandas Apply:  11%|█         | 906/8621 [00:42<06:27, 19.94it/s]



Pandas Apply:  11%|█         | 909/8621 [00:43<06:20, 20.28it/s]



Pandas Apply:  11%|█         | 912/8621 [00:43<06:12, 20.71it/s]



Pandas Apply:  11%|█         | 915/8621 [00:43<06:06, 21.04it/s]



Pandas Apply:  11%|█         | 918/8621 [00:43<06:02, 21.24it/s]



Pandas Apply:  11%|█         | 921/8621 [00:43<05:58, 21.49it/s]



Pandas Apply:  11%|█         | 924/8621 [00:43<05:57, 21.55it/s]



Pandas Apply:  11%|█         | 927/8621 [00:43<05:56, 21.56it/s]



Pandas Apply:  11%|█         | 930/8621 [00:44<05:57, 21.52it/s]



Pandas Apply:  11%|█         | 933/8621 [00:44<06:09, 20.79it/s]



Pandas Apply:  11%|█         | 936/8621 [00:44<0

Pandas Apply:  14%|█▎        | 1182/8621 [00:56<06:00, 20.65it/s]



Pandas Apply:  14%|█▎        | 1185/8621 [00:56<06:05, 20.34it/s]



Pandas Apply:  14%|█▍        | 1188/8621 [00:56<06:11, 19.99it/s]



Pandas Apply:  14%|█▍        | 1191/8621 [00:56<06:08, 20.16it/s]



Pandas Apply:  14%|█▍        | 1194/8621 [00:56<06:07, 20.20it/s]



Pandas Apply:  14%|█▍        | 1197/8621 [00:57<06:05, 20.31it/s]



Pandas Apply:  14%|█▍        | 1200/8621 [00:57<06:04, 20.36it/s]



Pandas Apply:  14%|█▍        | 1203/8621 [00:57<06:04, 20.33it/s]



Pandas Apply:  14%|█▍        | 1206/8621 [00:57<06:02, 20.46it/s]



Pandas Apply:  14%|█▍        | 1209/8621 [00:57<05:58, 20.68it/s]



Pandas Apply:  14%|█▍        | 1212/8621 [00:57<05:55, 20.84it/s]



Pandas Apply:  14%|█▍        | 1215/8621 [00:57<05:53, 20.95it/s]



Pandas Apply:  14%|█▍        | 1218/8621 [00:58<05:55, 20.84it/s]



Pandas Apply:  14%|█▍        | 1221/8621 [00:58<05:57, 20.72it/s]



Pandas Apply:  14%|█▍        | 122

...

Pandas Apply:  84%|████████▍ | 7232/8621 [05:55<01:02, 22.19it/s]



Pandas Apply:  84%|████████▍ | 7235/8621 [05:56<01:02, 22.11it/s]



Pandas Apply:  84%|████████▍ | 7238/8621 [05:56<01:03, 21.67it/s]



Pandas Apply:  84%|████████▍ | 7241/8621 [05:56<01:05, 21.13it/s]



Pandas Apply:  84%|████████▍ | 7244/8621 [05:56<01:06, 20.76it/s]



Pandas Apply:  84%|████████▍ | 7247/8621 [05:56<01:05, 20.86it/s]



Pandas Apply:  84%|████████▍ | 7250/8621 [05:56<01:06, 20.69it/s]



Pandas Apply:  84%|████████▍ | 7253/8621 [05:56<01:06, 20.63it/s]



Pandas Apply:  84%|████████▍ | 7256/8621 [05:57<01:05, 20.71it/s]



Pandas Apply:  84%|████████▍ | 7259/8621 [05:57<01:04, 20.96it/s]



Pandas Apply:  84%|████████▍ | 7262/8621 [05:57<01:04, 21.07it/s]



Pandas Apply:  84%|████████▍ | 7265/8621 [05:57<01:04, 20.98it/s]



Pandas Apply:  84%|████████▍ | 7268/8621 [05:57<01:04, 20.88it/s]



Pandas Apply:  84%|████████▍ | 7271/8621 [05:57<01:05, 20.74it/s]



Pandas Apply:  84%|████████▍ | 727

Pandas Apply:  87%|████████▋ | 7535/8621 [06:10<00:53, 20.22it/s]



Pandas Apply:  87%|████████▋ | 7538/8621 [06:10<00:53, 20.34it/s]



Pandas Apply:  87%|████████▋ | 7541/8621 [06:10<00:53, 20.17it/s]



Pandas Apply:  88%|████████▊ | 7544/8621 [06:10<00:53, 20.26it/s]



Pandas Apply:  88%|████████▊ | 7547/8621 [06:11<00:53, 20.25it/s]



Pandas Apply:  88%|████████▊ | 7550/8621 [06:11<00:53, 20.12it/s]



Pandas Apply:  88%|████████▊ | 7553/8621 [06:11<00:51, 20.55it/s]



Pandas Apply:  88%|████████▊ | 7556/8621 [06:11<00:51, 20.84it/s]



Pandas Apply:  88%|████████▊ | 7559/8621 [06:11<00:50, 20.87it/s]



Pandas Apply:  88%|████████▊ | 7562/8621 [06:11<00:50, 20.91it/s]



Pandas Apply:  88%|████████▊ | 7565/8621 [06:11<00:50, 20.80it/s]



Pandas Apply:  88%|████████▊ | 7568/8621 [06:12<00:50, 20.88it/s]



Pandas Apply:  88%|████████▊ | 7571/8621 [06:12<00:50, 20.62it/s]



Pandas Apply:  88%|████████▊ | 7574/8621 [06:12<00:50, 20.88it/s]



Pandas Apply:  88%|████████▊ | 757

Pandas Apply:  91%|█████████ | 7838/8621 [06:25<00:37, 21.00it/s]



Pandas Apply:  91%|█████████ | 7841/8621 [06:25<00:37, 20.91it/s]



Pandas Apply:  91%|█████████ | 7844/8621 [06:25<00:37, 20.99it/s]



Pandas Apply:  91%|█████████ | 7847/8621 [06:25<00:36, 20.96it/s]



Pandas Apply:  91%|█████████ | 7850/8621 [06:25<00:36, 20.90it/s]



Pandas Apply:  91%|█████████ | 7853/8621 [06:25<00:37, 20.55it/s]



Pandas Apply:  91%|█████████ | 7856/8621 [06:25<00:37, 20.66it/s]



Pandas Apply:  91%|█████████ | 7859/8621 [06:26<00:37, 20.48it/s]



Pandas Apply:  91%|█████████ | 7862/8621 [06:26<00:36, 20.53it/s]



Pandas Apply:  91%|█████████ | 7865/8621 [06:26<00:37, 20.09it/s]



Pandas Apply:  91%|█████████▏| 7868/8621 [06:26<00:36, 20.49it/s]



Pandas Apply:  91%|█████████▏| 7871/8621 [06:26<00:36, 20.72it/s]



Pandas Apply:  91%|█████████▏| 7874/8621 [06:26<00:36, 20.75it/s]



Pandas Apply:  91%|█████████▏| 7877/8621 [06:27<00:35, 20.84it/s]



Pandas Apply:  91%|█████████▏| 788

Pandas Apply:  93%|█████████▎| 8060/8621 [06:36<00:30, 18.65it/s]



Pandas Apply:  94%|█████████▎| 8062/8621 [06:36<00:30, 18.59it/s]



Pandas Apply:  94%|█████████▎| 8064/8621 [06:37<00:29, 18.62it/s]



Pandas Apply:  94%|█████████▎| 8066/8621 [06:37<00:30, 18.41it/s]



Pandas Apply:  94%|█████████▎| 8068/8621 [06:37<00:29, 18.49it/s]



Pandas Apply:  94%|█████████▎| 8070/8621 [06:37<00:29, 18.46it/s]



Pandas Apply:  94%|█████████▎| 8072/8621 [06:37<00:29, 18.49it/s]



Pandas Apply:  94%|█████████▎| 8074/8621 [06:37<00:29, 18.53it/s]



Pandas Apply:  94%|█████████▎| 8076/8621 [06:37<00:29, 18.58it/s]



Pandas Apply:  94%|█████████▎| 8078/8621 [06:37<00:29, 18.38it/s]



Pandas Apply:  94%|█████████▎| 8081/8621 [06:37<00:28, 19.03it/s]



Pandas Apply:  94%|█████████▍| 8083/8621 [06:38<00:28, 18.96it/s]



Pandas Apply:  94%|█████████▍| 8085/8621 [06:38<00:28, 19.09it/s]



Pandas Apply:  94%|█████████▍| 8087/8621 [06:38<00:28, 18.99it/s]



Pandas Apply:  94%|█████████▍| 808

Pandas Apply:  97%|█████████▋| 8339/8621 [06:50<00:13, 20.97it/s]



Pandas Apply:  97%|█████████▋| 8342/8621 [06:50<00:13, 20.90it/s]



Pandas Apply:  97%|█████████▋| 8345/8621 [06:50<00:13, 20.80it/s]



Pandas Apply:  97%|█████████▋| 8348/8621 [06:50<00:13, 20.61it/s]



Pandas Apply:  97%|█████████▋| 8351/8621 [06:51<00:13, 20.52it/s]



Pandas Apply:  97%|█████████▋| 8354/8621 [06:51<00:13, 20.37it/s]



Pandas Apply:  97%|█████████▋| 8357/8621 [06:51<00:13, 20.30it/s]



Pandas Apply:  97%|█████████▋| 8360/8621 [06:51<00:12, 20.32it/s]



Pandas Apply:  97%|█████████▋| 8363/8621 [06:51<00:12, 20.31it/s]



Pandas Apply:  97%|█████████▋| 8366/8621 [06:51<00:12, 20.26it/s]



Pandas Apply:  97%|█████████▋| 8369/8621 [06:51<00:12, 20.39it/s]



Pandas Apply:  97%|█████████▋| 8372/8621 [06:52<00:12, 20.66it/s]



Pandas Apply:  97%|█████████▋| 8375/8621 [06:52<00:11, 20.59it/s]



Pandas Apply:  97%|█████████▋| 8378/8621 [06:52<00:11, 20.75it/s]



Pandas Apply:  97%|█████████▋| 838

In [242]:
function_df = pd.read_csv("./classes")

In [243]:
function_filtered = pd.merge(train,function_df, how='left', left_on = 'function_dl', right_on = 'class1')

In [244]:
function_filtered['class1'] = clean_column(function_filtered['class1'], ["'", "(", ")", "/", "&", ","])

In [245]:
function_filtered['class2'] = clean_column(function_filtered['class2'], ["'", "(", ")", "/", "&", ","])

In [246]:
function_filtered['appended_class'] = function_filtered.apply(lambda x: x.class1 + " " + x.class2, axis=1)

In [247]:
function_filtered["cleaned"] = function_filtered.apply(lambda x: remove_string(x.filtered, x.appended_class), axis=1)

In [248]:
final1_data = function_filtered.filter(['article_name', 'cleaned', 'quantity', 'final_brand', 'class1', 'class2'])

In [249]:
final1_data.to_csv('./final1_data.csv', index=False)

In [250]:
final1_data = pd.read_csv('./final1_data.csv')

In [251]:
model = gensim.models.KeyedVectors.load_word2vec_format('./google.bin', binary=True)  



In [252]:
seed_choices = ['oranges', 'apples', 'avocado', 'onion', 'tomato', 'pants', 'shirt', 'pink', 'blue']
get_type("kitkat dessert delight truffle")

'truffle dessert'

In [253]:
final1_data["cleaned"] = final1_data.cleaned.fillna("")

In [254]:
final1_data['type'] = final1_data.swifter.apply(lambda x:get_type_specific(x.cleaned), axis=1)





Pandas Apply:   0%|          | 0/8621 [00:00<?, ?it/s]



Pandas Apply:   1%|          | 100/8621 [00:00<00:08, 989.99it/s]



Pandas Apply:   2%|▏         | 197/8621 [00:00<00:08, 982.47it/s]



Pandas Apply:   3%|▎         | 291/8621 [00:00<00:08, 966.45it/s]



Pandas Apply:   5%|▍         | 391/8621 [00:00<00:08, 974.14it/s]



Pandas Apply:   6%|▌         | 491/8621 [00:00<00:08, 978.43it/s]



Pandas Apply:   7%|▋         | 585/8621 [00:00<00:08, 965.71it/s]



Pandas Apply:   8%|▊         | 721/8621 [00:00<00:07, 1057.50it/s]



Pandas Apply:  10%|▉         | 859/8621 [00:00<00:06, 1136.97it/s]



Pandas Apply:  12%|█▏        | 1005/8621 [00:00<00:06, 1215.67it/s]



Pandas Apply:  13%|█▎        | 1150/8621 [00:01<00:05, 1274.80it/s]



Pandas Apply:  15%|█▍        | 1279/8621 [00:01<00:05, 1260.69it/s]



Pandas Apply:  16%|█▋        | 1419/8621 [00:01<00:05, 1297.83it/s]



Pandas Apply:  18%|█▊        | 1563/8621 [00:01<00:05, 1336.42it/s]



Pandas Apply:  20%|█▉        

In [255]:
final1_data

,article_name,cleaned,quantity,final_brand,class1,class2,type
0,chicken burger patty 250g,chicken burger patty,250g,venkys,ready to eat,branded food,flavor:chicken flavor:patty flavor:burger
1,britannia daily family long bread,daily family long bread,NaN,britannia,biscuits cookies crackers,branded food,flavor:bread
2,pooof potato kettle chips zesty masala pp 30g,pooof potato kettle zesty masala pp,30g,pooof!,chips namkeens snacks,branded food,flavor:zesty flavor:potato
3,fevibond tube 20 ml,tube,20 ml,fevibond,otc,beauty personal care,
4,kitkat dessert delight truffle 50g,kitkat dessert delight truffle,50g,nestle,chocolates cakes confectioneries,branded food,flavor:truffle flavor:dessert
5,blue bird spaghetti pasta 200g,spaghetti pasta,200g,blue bird,noodles pastas vermicelli,branded food,flavor:pasta
6,nirapara avalose podi 500gm,avalose podi,500gm,nirapara,toys,toys stationery,
7,nivea roll on fresh natural 50ml,roll on fresh natural,50ml,nivea,deos perfumes,beauty personal care,
8,fiama dw soap exotic dream bx 115g,dw soap exotic dream bx,115g,fiama di wills,skin care,beauty personal care,type:soap
9,sangis ktcn peri peri mayonnaise bt 200g,ktcn peri peri mayonnaise bt,200g,sangis kitchen,jams spreads honey,branded food,type:mayonnaise flavor:mayonnaise


In [256]:
final1_data['cleaned_more'] = final1_data.swifter.apply(lambda x: remove_string(x.cleaned, x.type), axis=1)





Pandas Apply:   0%|          | 0/8621 [00:00<?, ?it/s]



Pandas Apply:  48%|████▊     | 4098/8621 [00:00<00:00, 40974.28it/s]



Pandas Apply:  96%|█████████▋| 8304/8621 [00:00<00:00, 41293.90it/s]



Pandas Apply: 100%|██████████| 8621/8621 [00:00<00:00, 40879.31it/s]

In [257]:
final1_data

,article_name,cleaned,quantity,final_brand,class1,class2,type,cleaned_more
0,chicken burger patty 250g,chicken burger patty,250g,venkys,ready to eat,branded food,flavor:chicken flavor:patty flavor:burger,chicken burger patty
1,britannia daily family long bread,daily family long bread,NaN,britannia,biscuits cookies crackers,branded food,flavor:bread,daily family long bread
2,pooof potato kettle chips zesty masala pp 30g,pooof potato kettle zesty masala pp,30g,pooof!,chips namkeens snacks,branded food,flavor:zesty flavor:potato,pooof potato kettle zesty masala pp
3,fevibond tube 20 ml,tube,20 ml,fevibond,otc,beauty personal care,,tube
4,kitkat dessert delight truffle 50g,kitkat dessert delight truffle,50g,nestle,chocolates cakes confectioneries,branded food,flavor:truffle flavor:dessert,kitkat dessert delight truffle
5,blue bird spaghetti pasta 200g,spaghetti pasta,200g,blue bird,noodles pastas vermicelli,branded food,flavor:pasta,spaghetti pasta
6,nirapara avalose podi 500gm,avalose podi,500gm,nirapara,toys,toys stationery,,avaloe podi
7,nivea roll on fresh natural 50ml,roll on fresh natural,50ml,nivea,deos perfumes,beauty personal care,,roll on freh natural
8,fiama dw soap exotic dream bx 115g,dw soap exotic dream bx,115g,fiama di wills,skin care,beauty personal care,type:soap,dw soap exotic dream bx
9,sangis ktcn peri peri mayonnaise bt 200g,ktcn peri peri mayonnaise bt,200g,sangis kitchen,jams spreads honey,branded food,type:mayonnaise flavor:mayonnaise,ktcn peri peri mayonnaise bt


In [258]:
stopwords = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]

In [259]:
final1_data["get_cleaned"] = final1_data.swifter.apply(lambda x: get_words_from_sentence(x.cleaned_more), axis=1)





Pandas Apply:   0%|          | 0/8621 [00:00<?, ?it/s]



Pandas Apply:  38%|███▊      | 3315/8621 [00:00<00:00, 33147.66it/s]



Pandas Apply:  78%|███████▊  | 6751/8621 [00:00<00:00, 33499.42it/s]



Pandas Apply: 100%|██████████| 8621/8621 [00:00<00:00, 33142.92it/s]

In [260]:
final1_data.to_csv("./final_datav1.1.1.csv", index=False)

In [261]:
final1_data.head(100)

,article_name,cleaned,quantity,final_brand,class1,class2,type,cleaned_more,get_cleaned
0,chicken burger patty 250g,chicken burger patty,250g,venkys,ready to eat,branded food,flavor:chicken flavor:patty flavor:burger,chicken burger patty,"[chicken, burger, patty]"
1,britannia daily family long bread,daily family long bread,NaN,britannia,biscuits cookies crackers,branded food,flavor:bread,daily family long bread,"[daily, family, long, bread]"
2,pooof potato kettle chips zesty masala pp 30g,pooof potato kettle zesty masala pp,30g,pooof!,chips namkeens snacks,branded food,flavor:zesty flavor:potato,pooof potato kettle zesty masala pp,"[potato, kettle, masala]"
3,fevibond tube 20 ml,tube,20 ml,fevibond,otc,beauty personal care,,tube,[tube]
4,kitkat dessert delight truffle 50g,kitkat dessert delight truffle,50g,nestle,chocolates cakes confectioneries,branded food,flavor:truffle flavor:dessert,kitkat dessert delight truffle,"[dessert, delight, truffle]"
5,blue bird spaghetti pasta 200g,spaghetti pasta,200g,blue bird,noodles pastas vermicelli,branded food,flavor:pasta,spaghetti pasta,"[spaghetti, pasta]"
6,nirapara avalose podi 500gm,avalose podi,500gm,nirapara,toys,toys stationery,,avaloe podi,[]
7,nivea roll on fresh natural 50ml,roll on fresh natural,50ml,nivea,deos perfumes,beauty personal care,,roll on freh natural,"[roll, natural]"
8,fiama dw soap exotic dream bx 115g,dw soap exotic dream bx,115g,fiama di wills,skin care,beauty personal care,type:soap,dw soap exotic dream bx,"[soap, exotic, dream]"
9,sangis ktcn peri peri mayonnaise bt 200g,ktcn peri peri mayonnaise bt,200g,sangis kitchen,jams spreads honey,branded food,type:mayonnaise flavor:mayonnaise,ktcn peri peri mayonnaise bt,"[peri, peri, mayonnaise]"


In [262]:
output = final1_data.filter(['article_name', 'quantity', 'final_brand', 'class1', 'class2', 'type', 'get_cleaned'])

In [263]:
output = output.rename(columns={"final_brand": "brand", "class1": "function", "class2": "function_parent", "type": "type-flavour-colour", "get_cleaned": "misc"})

In [264]:
output["misc"] = output.misc.swifter.apply(lambda x: ",".join(x))





Pandas Apply:   0%|          | 0/8621 [00:00<?, ?it/s]



Pandas Apply: 100%|██████████| 8621/8621 [00:00<00:00, 815937.69it/s]

In [265]:
output.function = output.swifter.apply(lambda x: x.function + " | " + x.function_parent, axis=1)

In [266]:
del output["function_parent"]

In [267]:
output.to_csv("./outputv3.csv", index=False)

In [268]:
output

,article_name,quantity,brand,function,type-flavour-colour,misc
0,chicken burger patty 250g,250g,venkys,ready to eat | branded food,flavor:chicken flavor:patty flavor:burger,"chicken,burger,patty"
1,britannia daily family long bread,NaN,britannia,biscuits cookies crackers | branded food,flavor:bread,"daily,family,long,bread"
2,pooof potato kettle chips zesty masala pp 30g,30g,pooof!,chips namkeens snacks | branded food,flavor:zesty flavor:potato,"potato,kettle,masala"
3,fevibond tube 20 ml,20 ml,fevibond,otc | beauty personal care,,tube
4,kitkat dessert delight truffle 50g,50g,nestle,chocolates cakes confectioneries | branded food,flavor:truffle flavor:dessert,"dessert,delight,truffle"
5,blue bird spaghetti pasta 200g,200g,blue bird,noodles pastas vermicelli | branded food,flavor:pasta,"spaghetti,pasta"
6,nirapara avalose podi 500gm,500gm,nirapara,toys | toys stationery,,
7,nivea roll on fresh natural 50ml,50ml,nivea,deos perfumes | beauty personal care,,"roll,natural"
8,fiama dw soap exotic dream bx 115g,115g,fiama di wills,skin care | beauty personal care,type:soap,"soap,exotic,dream"
9,sangis ktcn peri peri mayonnaise bt 200g,200g,sangis kitchen,jams spreads honey | branded food,type:mayonnaise flavor:mayonnaise,"peri,peri,mayonnaise"
